In [0]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content')


!unzip -q drive/My\ Drive/noCropped_dataset.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import cv2
import random

import numpy as np
import keras
from PIL import Image

from keras.layers.pooling import MaxPooling2D,AveragePooling2D
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint 
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler 

#choose the images size
img_size_row=224
img_size_column=224
global IMAGE_SIZE
IMAGE_SIZE=(img_size_row , img_size_column)

#configuration for training
train_size= 9766
test_size= 1487
val_size= 2764

#choose the batch size
batch_size=64
steps_per_epoch=train_size*5/batch_size
validation_steps=val_size*5/batch_size

#choose the epochs
epochs=50

#num of classes to be trained
num_classes=32

In [0]:
import tensorflow as tf

def data_aug(img):
  #Brightess
  if random.randint(0,1):    
     exposure = random.randint(1,2)
     if exposure==2:
         brightess=img*0.7
     else:
         brightess=img*1.4
     brightess = np.clip(brightess, 0, 255)
     img = brightess.astype(np.uint8)

  #Gaussian noise
  if random.randint(0,1):
     row,col,ch= img.shape
     binom=np.random.binomial(n=500, p=0.5,size=(row,col,ch))
     binom=binom-250
     noisy = img + binom
     noisy = np.clip(noisy, 0, 255)
     img = noisy.astype(np.uint8)

  #Flip
  if random.randint(0,1):
     flip=np.fliplr(img)
     img=flip

  return img

In [0]:
def generator(path,txt,num_classes,batch_size=16):
  f = open(txt,"r")
  
  ds=[]
  for row in f:
    img, classe, s = row.split(";",3)
    ds.append( (img,classe) )

  dataset=np.array(ds)

  while True:
    np.random.shuffle(dataset)
    img1_list=[]
    label = []

    for i in dataset:
      img_1 = i[0]

      img_1 = cv2.imread(path+"/"+img_1)

      if(img_1 is None):
        print(img)

      img_1 = data_aug(np.array(img_1))
      img_1 = cv2.resize(img_1, (224,224) ,0,0, cv2.INTER_LINEAR)
      img_1 = img_1.astype(np.float32)
      img_1 /= 255
      img1_list.append(img_1)
      label.append(int(i[1]))

      if len(label)==batch_size:
        array1 = np.array(img1_list)
        label_categorical=keras.utils.to_categorical(label,num_classes=num_classes)
        lab = np.array(label_categorical)
        img1_list=[]
        label = []
        yield (array1 , lab)          

In [0]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, num_classes):
    x = base_model.output
    x = AveragePooling2D((5, 5), name='avg_pool')(x)
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    predictions = Dense(num_classes, activation='softmax', name='finalfc')(x)
    finetune_model = Model(inputs=base_model.input, outputs=predictions)
    return finetune_model

dropout = 0.25

base_model = keras.applications.mobilenet.MobileNet(weights='imagenet',
                            include_top=False,
                            input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1], 3))

finetune_model = build_finetune_model(base_model,
                                      dropout=dropout,
                                      num_classes=num_classes)

In [0]:
learning_rate_di_base=0.0001
learning_rate_decay_factor=0.05
learning_rate_decay_epochs=10
def step_decay_schedule(initial_lr=learning_rate_di_base, decay_factor=learning_rate_decay_factor, step_size=learning_rate_decay_epochs):
    '''
    Wrapper function to create a LearningRateScheduler with step decay schedule.
    '''
    def schedule(epoch):
        return initial_lr * (decay_factor ** np.floor(epoch/step_size))
    
    return LearningRateScheduler(schedule,verbose=1)

In [0]:
#train
finetune_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#callbacks
filepath="MobileNet_NoDetection.hdf5"
lr_sched = step_decay_schedule()
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [EarlyStopping(monitor='val_acc',
                           patience=10,
                           verbose=1,
                           min_delta=1e-4,
                           mode='max'),
                  lr_sched,
                  checkpoint]
print(finetune_model.summary())


In [0]:
path="noCropped_dataset"

t_gen = generator(path,"train.txt",num_classes,batch_size=batch_size)
v_gen = generator(path,"val.txt",num_classes,batch_size=batch_size)

In [0]:

finetune_model.fit_generator(t_gen,steps_per_epoch=steps_per_epoch , epochs=epochs, verbose=1,callbacks=callbacks_list,
                               validation_data=v_gen, validation_steps=validation_steps,use_multiprocessing=True,workers = 4,initial_epoch = 0)